In [ ]:
import os
import random

# Output folder for KPG instances
folder = "C:/Users/hyunwoolee/OneDrive - Virginia Tech/Hyunwoo Research/BZR_KPG/KPG_generated"

def generate_kpg_instances():
    """
    Generates synthetic instances for the Knapsack Potential Game (KPG).

    This implementation is adapted from the original C++ version by 
    Dragotto and Scatamacchia (2023), available at:
    https://github.com/gdragotto/ZeroRegretsAlgorithm/blob/main/instance_generators/instances_kp/generator.cpp

    Key differences:
        - For type C (interaction_type = 2), the interaction terms C_ij are drawn from a **reduced range** [-R/5, 0]
          instead of the original [-R, R], to ensure feasibility in large-scale games.

    Interaction Types:
        - 0: Potential game (shared values per player).
        - 1: Independent, non-negative C_ij values.
        - 2: Negative C_ij values ∈ [-R/5, 0] (for strategic complements or disincentives).

    Each instance is written to a .txt file with format:
        - Line 1: number of players and number of items
        - Line 2: budget per player
        - Remaining lines: item index followed by profits, weights, and pairwise interactions
    """

    # Parameter grid
    player_sizes = [2, 3, 5, 8, 10, 15, 20, 25, 30]
    item_sizes = [100]  # Extendable: [25, 50, 100, 150]
    budget_ratios = [2, 5, 8]  # Interpreted as 0.2, 0.5, 0.8
    interaction_types = [0, 1, 2]  # A, B, C

    R = 100  # Range for profits and weights

    for n in player_sizes:
        for interaction_type in interaction_types:
            for m in item_sizes:
                for ratio in budget_ratios:

                    random.seed()  # Seed from system clock

                    sum_weights = [0] * n
                    budgets = [0] * n
                    CPotential = [random.randint(1, R) for _ in range(n)]
                    items = []

                    for j in range(m):
                        item_row = []

                        # Generate profit and weight per player
                        for i in range(n):
                            profit = random.randint(1, R)
                            weight = random.randint(1, R)
                            item_row.extend([profit, weight])
                            sum_weights[i] += weight

                        # Generate interaction terms for player pairs
                        for p in range(n):
                            for o in range(n):
                                if p != o:
                                    if interaction_type == 0:
                                        interaction = CPotential[p]
                                    elif interaction_type == 1:
                                        interaction = random.randint(1, R)
                                    elif interaction_type == 2:
                                        interaction = -random.randint(1, R // 5)  # Reduced range for feasibility
                                    else:
                                        raise ValueError(f"Unsupported interaction type: {interaction_type}")
                                    item_row.append(interaction)

                        items.append(item_row)

                    # File and folder setup
                    interaction_label = {0: "pot", 1: "cij", 2: "cij-sn"}[interaction_type]
                    category_letter = {0: "A", 1: "B", 2: "C"}[interaction_type]
                    instance_name = f"{n}-{m}-{ratio}-{interaction_label}.txt"
                    output_dir = os.path.join(folder, f"type_{category_letter}")
                    os.makedirs(output_dir, exist_ok=True)
                    output_path = os.path.join(output_dir, instance_name)

                    # Compute budget per player based on total weights
                    budgets = [round((ratio / 10) * total) for total in sum_weights]

                    # Write the instance file
                    with open(output_path, "w") as f:
                        f.write(f"{n} {m}\n")
                        f.write(" ".join(map(str, budgets)) + "\n")
                        for j, item_row in enumerate(items):
                            f.write(f"{j} {' '.join(map(str, item_row))}\n")

if __name__ == "__main__":
    generate_kpg_instances()
